# Employee Workload Balancer
You are managing a team's workload and need to optimize task distribution. Each employee is assigned a list of tasks with varying levels of difficulty. your goal is to:
1. Filter out tasks that are lebeled as 'Low Priority'.
2. Calculate the total difficulty_score for all remaining tasks per employee.
3. Scale the difficulty_score by reducing it by 10%.
4. Sort employees by their scaled total difficulty in descending order.
5. Create a generator to yield each employee's name and their adjusted total difficulty.

In [15]:
employees = [
    {"name": "Alice", 
     "tasks": 
        [{"name": "Task 1", "difficulty": 5, "priority": "High"}, 
         {"name": "Task 2", "difficulty": 2, "priority": "Low"}]},
    {"name": "Bob", 
     "tasks": 
        [{"name": "Task 3", "difficulty": 4, "priority": "Medium"}, 
         {"name": "Task 4", "difficulty": 3, "priority": "High"}]},
    {"name": "Charlie", 
     "tasks": 
        [{"name": "Task 5", "difficulty": 1, "priority": "Low"}, 
        {"name": "Task 6", "difficulty": 6, "priority": "High"}]}
]

In [16]:
from functools import reduce
from typing import List, Dict, Any

# Helper function to scale difficuty
def scale_task_difficulty(tasks: List[Dict[str, Any]], factor: float) -> List[Dict[str, Any]]:
    return list(map(lambda t:{ **t,
                              'difficulty': t['difficulty']*factor
                            },
                    tasks))

# Helper function to calculate difficulty_score
def calculate_difficulty_score(tasks: List[Dict[str, Any]]) -> float:
    return reduce(lambda acc, t: acc+t['difficulty'], tasks, 0) if tasks else 0.0

# Helper function to filter tasks by parameter
def filter_tasks_by_parameter(tasks: List[Dict[str, Any]], filter_factor: str) -> List[Dict[str, Any]]:
    return [t for t in tasks if t['priority']!=filter_factor]
    # return list(filter(lambda t: t['priority']!=filter_factor, tasks))

# 1. Filter employees by priority 
employees_with_filtered_tasks = list(map(
    lambda e: {**e,
                'tasks': filter_tasks_by_parameter(e['tasks'],'Low')
                }, 
    employees
    ))

# 2. Calculate difficult for the remaining tasks
employees_with_scores = list(map(
    lambda e: {**e,
                'difficulty_score': calculate_difficulty_score(e['tasks'])},
                employees_with_filtered_tasks
))

# 3. Apply a scaling factor 
scaling_factor = 0.9
scaled_employees = list(map(
    lambda e: {
        **e, 
        'difficulty_score': e['difficulty_score']*scaling_factor,
        'tasks': scale_task_difficulty(e['tasks'], scaling_factor),
    },
    employees_with_scores
))

# 4. Sort employees by difficulty_score in descendent order
sorted_employees = sorted(
    scaled_employees,
    key = lambda e: (e['difficulty_score']),
    reverse=True
) 

# 5. Generator to yield employee's name & difficulty score
def employee_generator(employees):
    for employee in employees:
        yield f'Employee: {employee['name']}, Difficulty Score: {employee['difficulty_score']:.2f}'

for employ_detail in employee_generator(sorted_employees):
    print(employ_detail)


Employee: Bob, Difficulty Score: 6.30
Employee: Charlie, Difficulty Score: 5.40
Employee: Alice, Difficulty Score: 4.50


# Dynamic Resource Allocation for Cloud Servers
You are managing a pool of cloud servers and need to optimize resource allocation based on current loads. Each server has a set of running tasks, and your goal is to:
1. Filter out tasks that consume less than specific amount of memory (e.g., 1GB).
2. Calculate the total memory usage for each  server.
3. Apply a scaling factor to reduce memory consumption for optimization (e.g., reduce memory usage by 15%).
4. Sort servers by their scaled memory usage in ascending order.
5. Create a generator that yilds the server Id, the count of tasks on the server, and the scaled memory usage.

In [19]:
servers = [
    {"id": "S1", "tasks": [
        {"name": "Task A", "memory_gb": 2.5}, 
        {"name": "Task B", "memory_gb": 0.8}, 
        {"name": "Task C", "memory_gb": 1.5}]},
    {"id": "S2", "tasks": [
        {"name": "Task D", "memory_gb": 3.0}, 
        {"name": "Task E", "memory_gb": 1.2}]},
    {"id": "S3", "tasks": [
        {"name": "Task F", "memory_gb": 0.5}, 
        {"name": "Task G", "memory_gb": 2.0}, 
        {"name": "Task H", "memory_gb": 1.8}]}
]

In [22]:
from functools import reduce
from typing import List, Dict, Any

# Helper funcion to filter task by parameter
def filter_tasks_by_memory(tasks: List[Dict[str, Any]], min_memory:float)-> List[Dict[str,any]]:
    """Filters tasks consuming more than the specific memory."""
    return list(filter(lambda t: t['memory_gb'] > min_memory, tasks))
    # return [t for t in tasks if t['memory_gb'] > min_memory]

# Helper function to calculate total memory
def calculate_total_memory(tasks: List[Dict[str, Any]])->float:
    """Calculates the total memory usage of given tasks."""
    return reduce(lambda acc, t: acc+t['memory_gb'], tasks, 0) if tasks else 0.0
    return sum(task['memory_gb'] for task in tasks) if tasks else 0.0

# Helper function to scale memory
def scale_memory(memory: float, factor: float) -> float:
    return round(memory * factor, 2)

# Pipline for functional processing
def process_servers(servers: List[Dict[str, Any]], min_memory:float, scaling_factor:float)-> List[str]:
    return list(map(
        lambda server: f'Server: {server['id']}, Task Count: {len(server['tasks'])}, Scaled memory: {server['scaled_memory']} GB.',
        sorted(
            map(
                lambda s: {
                    **s,
                    'tasks': filter_tasks_by_memory(s['tasks'], min_memory),
                    'memory': calculate_total_memory(s['tasks']),
                    'scaled_memory': scale_memory(calculate_total_memory(s['tasks']), scaling_factor) 
                }, servers
            ), key=lambda s: s['scaled_memory']
        )
    ))

for result in process_servers(servers, 1.0, 0.85):
    print(result)


# 1. Filter tasks that consume less specific amount
minimum_usage = 1.0
filtered_tasks = list(map(lambda s:{**s, 'tasks': filter_tasks_by_memory(s['tasks'], minimum_usage)}, servers))

# 2. Calculate total memory for each server
memory_server = list(map(lambda s:{**s,'memory': calculate_total_memory(s['tasks'])}, filtered_tasks))

# 3. Apply scaling factor to memory
scaling_factor = 0.85
scaled_memory = list(map(lambda s: {**s,'scaled memory': round(s['memory']*scaling_factor, 2)}, memory_server))

# 4. Sort server by scaled_memory
sorted_scaled_memory = sorted(scaled_memory,key = lambda s: (s['scaled memory']) )

# 5. Generator to yield server details
def server_generator(servers:List[Dict[str, Any]]):
    for server in servers:
        yield f'Server: {server['id']}, Task Count: {len(server['tasks'])}, Scaled Memory: {server['scaled memory']:.2f} GB'

# Print server details
for server in server_generator(sorted_scaled_memory):
    print(server)

Server: S2, Task Count: 2, Scaled memory: 3.57 GB.
Server: S3, Task Count: 2, Scaled memory: 3.65 GB.
Server: S1, Task Count: 2, Scaled memory: 4.08 GB.
Server: S3, Task Count: 2, Scaled Memory: 3.23 GB
Server: S1, Task Count: 2, Scaled Memory: 3.40 GB
Server: S2, Task Count: 2, Scaled Memory: 3.57 GB
